In [3]:
import ccxt
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import subprocess
import re

In [4]:
# Function to fetch top N cryptocurrencies by market capitalization
def get_top_cryptos(n=15):
    exchange = ccxt.binance()  # You can change this to the desired exchange
    markets = exchange.fetch_markets()
    sorted_markets = sorted(markets, key=lambda x: x['info']['market_cap_usd'] if 'market_cap_usd' in x['info'] else 0, reverse=True)[:n]
    top_symbols = [market['symbol'] for market in sorted_markets]
    return top_symbols



In [8]:
# Initialize the CCXT exchange object
exchange = ccxt.binance()  # You can change this to the desired exchange

# Get top 10 cryptocurrencies and major forex symbols
crypto_symbols = get_top_cryptos()
all_symbols = crypto_symbols

In [12]:
# Iterate through symbols
for symbol in all_symbols:
    # Define the trading pair and timeframe
    timeframe = '1d'

    # Fetch historical OHLCV data
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=30)

    # Convert the data to a Pandas DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Detect engulfing pattern and pinbar pattern
    df['engulfing'] = 0
    df['pinbar']    = 0
    
    pinbar_intensity_wick = 3

    for i in range(1, len(df)):
        if (df['close'][i] > df['open'][i]) and (df['close'][i-1] < df['open'][i-1]) and (df['close'][i] > df['high'][i-1]):
            df.loc[i, 'engulfing'] = 1  # Bullish engulfing
            
        elif (df['close'][i] < df['open'][i]) and (df['close'][i-1] > df['open'][i-1]) and (df['close'][i] < df['low'][i-1]) :
            df.loc[i, 'engulfing'] = -1  # Bearish engulfing
            
        if ( abs(df['open'][i] - df['low'][i]) > pinbar_intensity_wick * abs(df['open'][i] - df['close'][i]) and
            abs(df['open'][i] - df['low'][i] > pinbar_intensity_wick *abs(df['close'][i] - df['high'][i]))):
            df.loc[i, 'pinbar'] = 1  # Bullish pinbar
            
        elif ( abs(df['close'][i] - df['low'][i]) > pinbar_intensity_wick * abs(df['open'][i] - df['close'][i]) and
            abs(df['close'][i] - df['low'][i] > pinbar_intensity_wick *abs(df['open'][i] - df['high'][i]))):
            df.loc[i, 'pinbar'] = 1  # Bullish pinbar
            
        elif ( abs(df['close'][i] - df['high'][i]) > pinbar_intensity_wick * abs(df['open'][i] - df['close'][i]) and
            pinbar_intensity_wick * abs(df['open'][i] - df['low'][i] < abs(df['close'][i] - df['high'][i]))):
            df.loc[i, 'pinbar'] = -1  # Bearish pinbar
            
        elif ( abs(df['open'][i] - df['high'][i]) > pinbar_intensity_wick * abs(df['open'][i] - df['close'][i]) and
            pinbar_intensity_wick * abs(df['close'][i] - df['low'][i] < abs(df['open'][i] - df['high'][i]))):
            df.loc[i, 'pinbar'] = -1  # Bearish pinbar
            
    
    # Filter engulfing and pin bar patterns
    pattern_df = df[(df['engulfing'] != 0) | (df['pinbar'] != 0)]

    # Plot the data using a candlestick chart
    fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                                         open=df['open'],
                                         high=df['high'],
                                         low=df['low'],
                                         close=df['close'],
                                         increasing_line_color='green',
                                         decreasing_line_color='red'
                                        )])

    # Add arrows for engulfing and pin bar patterns
    for index, row in pattern_df.iterrows():
        color = 'blue' if row['engulfing'] == 1 else 'red' if row['engulfing'] == -1 else 'green' if row['pinbar'] == 1 else 'purple'
        arrow_direction = 'up' if row['engulfing'] == 1 or row['pinbar'] == 1 else 'down'
        annotation_text = 'Engulfing' if row['engulfing'] != 0 else 'Pin Bar'

        fig.add_annotation(
            dict(x=row['timestamp'],
                 y=row['low'] if arrow_direction == 'up' else row['high'],
                 text=annotation_text,
                 arrowhead=2,
                 arrowsize=1,
                 arrowwidth=2,
                 arrowcolor=color,
                 ax=-5,
                 ay=+30 if arrow_direction == 'up' else -30,
                 opacity=0.7
                )
        )

    
    fig.update_layout(title=f'{symbol} Engulfing/Pinbar Pattern Detection',
                      xaxis_title='Timestamp',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)


    # Show the plot (optional)
    fig.show()
